Create and Test Team Ratings for 2023 Season - Round by Round

In [1]:
import pandas as pd
import re
import numpy as np
from scipy import stats
from sklearn import metrics

import os
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option('display.precision', 4)

In [2]:
player_stats = pd.read_csv("/Users/ciaran/Documents/Projects/AFL/data/scored_player_stats.csv")
player_stats['Season'] = player_stats['Match_ID'].apply(lambda x: int(x[:4]))
player_stats['Player_Season'] = player_stats['Player'] + "_" + player_stats['Season'].astype(str)

player_stats['Score'] = player_stats['Behinds'] + player_stats['Goals']*6
player_stats['xScore_Diff'] = player_stats['Score'] - player_stats['xScore']

player_stats['Home_Team'] = player_stats['Match_ID'].apply(lambda x: x.split("_")[1]).str.replace(r"([A-Z])", r" \1").str.strip()
player_stats['Away_Team'] = player_stats['Match_ID'].apply(lambda x: x.split("_")[-1]).str.replace(r"([A-Z])", r" \1").str.strip()
player_stats['Opponent'] = np.where(player_stats['Team'] == player_stats['Home_Team'], player_stats['Away_Team'], player_stats['Home_Team'])

finals_round_map = {'F1':24,
                    'F2':25,
                    'F3':26,
                    'F4':27}
player_stats['Round_ID'] = player_stats['Round_ID'].astype(str)
player_stats['Round'] = (player_stats['Round_ID'].str[4:].replace(finals_round_map)).astype(int)
player_stats['Round_str'] = player_stats['Round_ID'].str[4:].replace(finals_round_map)
player_stats['Round_ID_num'] = (player_stats['Season'].astype(str) + player_stats['Round'].astype(str)).astype(int)

player_stats.tail()

,Match_ID,Team,Player,Round_ID,AFL_API_Player_ID,Player_Type,playerId,Age,Height,Weight,Number,Kicking_Foot,State_Of_Origin,Draft_Year,Debut_Year,Recruited_From,Draft_Position,Draft_Type,Photo_URL,Date_Of_Birth,Percent_Played,Behinds,Bounces,Centre_Bounces_Attended,Centre_Clearances,Clangers,Defensive_Contest_Losses,Defensive_Contest_Loss_Percentage,Defensive_One_On_One_Contests,Contested_Marks,Contested_Possession_Rate,Contested_Possessions,Offensive_One_On_One_Contests,Offensive_Contest_Wins,Offensive_Contest_Win_Percentage,Defensive_Half_Pressure_Acts,Disposal_Efficiency,Disposals,AFL_Fantasy_Points,Effective_Disposals,Effective_Kicks,Inside_50_Ground_Ball_Gets,Frees_Against,Frees_For,Goal_Accuracy,Goal_Assists,Goals,Ground_Ball_Gets,Handballs,Hit_Outs,Hit_Outs_To_Advantage,Hit_Outs_To_Advantage_Rate,Hit_Out_Win_Percentage,Inside_50s,Intercept_Marks,Intercepts,Kick_Efficiency,Kick_Ins,Kick_Ins_Played_On,Kicks,Kick_To_Handball_Ratio,Marks,Marks_Inside_50,Marks_On_Lead,Metres_Gained,One_Percenters,Pressure_Acts,Player_Rating_Points,Rebound_50s,Ruck_Contests,Score_Involvements,Score_Launches,Shots_At_Goal,Spoils,Stoppage_Clearances,Tackles,Tackles_Inside_50,Clearances,Possessions,Turnovers,Uncontested_Possessions,AFLCA_Player_ID,Coaches_Votes,Position,Team_Status,Position_Sub_Group,Position_Group,Year,Brownlow_Votes,Season,xScore,xT_created,xT_denied,vaep_value,offensive_value,defensive_value,exp_vaep_value,exp_offensive_value,exp_defensive_value,xT_received,xT_prevented,vaep_value_received,exp_vaep_value_received,Player_Season,Score,xScore_Diff,Home_Team,Away_Team,Opponent,Round,Round_str,Round_ID_num
27846,202323_WesternBulldogs_WestCoast,Western Bulldogs,Oskar Baker,202323,Oskar_Baker,MIDFIELDER,CD_I1008855,25,184,87,13,RIGHT,QLD,2017.0,2019.0,Wilston Grange (Qld)/Aspley (NEAFL)/Melbourne,48.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1998-05-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Half Forward Flank Right,FINAL_TEAM,Half-Forward,Forward,NaN,NaN,2023,7.9598,-0.3226,0.0167,0.3991,0.7006,-0.3015,2.8523,3.8194,-0.9671,-0.3267,0.0713,1.0483,3.4625,Oskar Baker_2023,NaN,NaN,Western Bulldogs,West Coast,West Coast,23,23,202323
27847,202323_WesternBulldogs_WestCoast,Western Bulldogs,Rhylee West,202323,Rhylee_West,MEDIUM_FORWARD,CD_I1006127,23,183,82,14,RIGHT,VIC,2018.0,2019.0,Strathmore (Vic)/St Kevin&apos;s College (Vic)...,26.0,fatherSon,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,2000-07-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Forward Pocket Right,FINAL_TEAM,Forward-Pocket,Forward,NaN,NaN,2023,9.7706,0.1389,0.0000,0.5694,0.6452,-0.0758,7.0749,7.4761,-0.4012,0.4868,0.0043,0.9269,6.2204,Rhylee West_2023,NaN,NaN,Western Bulldogs,West Coast,West Coast,23,23,202323
27848,202323_WesternBulldogs_WestCoast,Western Bulldogs,Rory Lobb,202323,Rory_Lobb,KEY_FORWARD,CD_I990740,30,207,106,7,RIGHT,WA,2013.0,2014.0,Bassendean (WA)/Swan Districts (WAFL)/GWS/Frem...,29.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1993-02-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Forward Pocket Left,FINAL_TEAM,Forward-Pocket,Forward,NaN,NaN,2023,17.1166,-0.1439,-0.0025,1.7295,1.8518,-0.1224,11.1693,12.7523,-1.5830,0.4878,-0.0002,1.1501,10.5511,Rory Lobb_2023,NaN,NaN,Western Bulldogs,West Coast,West Coast,23,23,202323
27849,202323_WesternBulldogs_WestCoast,Western Bulldogs,Taylor Duryea,

Pre-Season

In [3]:
def get_previous_round_id(round_id):
    
    finals_round_map = {'F1':24,
                        'F2':25,
                        'F3':26,
                        'F4':27}
    
    season = round_id[:4]
    if 'F' in round_id:
        round_num = finals_round_map[round_id[4:]]
    else:
        round_num = round_id[4:]
    
    previous_round_num = int(round_id[4:])-1
    previous_round_num = str(previous_round_num).zfill(2)
    if previous_round_num == 1:
        previous_season = int(season) - 1
    else:
        previous_season = season
    previous_round_id = str(previous_season) + str(previous_round_num)
    
    return previous_round_id

In [4]:
def get_score_standard_deviation(data):
    return data.groupby(['Match_ID', 'Team']).sum()['Score'].std()

In [5]:
def create_team_summary(player_stats, rounds = None):
    
    player_stats_copy = player_stats.copy()
    
    if rounds is not None:
        all_rounds = sorted(list(set(player_stats_copy['Round_ID'])))
        rating_rounds = all_rounds[-rounds:]
        player_stats_copy = player_stats_copy[player_stats_copy['Round_ID'].isin(rating_rounds)]
    
    team_for = player_stats_copy.groupby(['Team']).sum()[['Score', 'xScore','vaep_value', 'vaep_value_received', 'offensive_value', 'defensive_value', 'exp_vaep_value', 'exp_vaep_value_received', 'exp_offensive_value', 'exp_defensive_value']]
    team_for.columns = [x+"_F" for x in list(team_for)]
    team_against = player_stats_copy.groupby(['Opponent']).sum()[['Score', 'xScore', 'vaep_value', 'vaep_value_received', 'offensive_value', 'defensive_value', 'exp_vaep_value', 'exp_vaep_value_received', 'exp_offensive_value', 'exp_defensive_value']]
    team_against.columns = [x+"_A" for x in list(team_against)]
    team_games = player_stats_copy.groupby(['Match_ID', 'Team']).size().reset_index().groupby(['Team']).count()[['Match_ID']]
    team_games.columns = ['Games']
    team_summary = pd.concat([team_games, team_for, team_against], axis=1)
    
    for col in [x.replace("_F", "") for x in list(team_for)]:
        team_summary[col+"_pergame_F"] = team_summary[col+"_F"] / team_summary["Games"]
        team_summary[col+"_pergame_A"] = team_summary[col+"_A"] / team_summary["Games"]

        team_summary[col+"_diff"] = team_summary[col+"_F"] - team_summary[col+"_A"]
        team_summary[col+"_pergame_diff"] = team_summary[col+"_pergame_F"] - team_summary[col+"_pergame_A"]

    return team_summary

In [6]:
def get_teams(match_id):
    
    home_team = re.sub(r"\B([A-Z])", r" \1", match_id.split("_")[1])
    away_team = re.sub(r"\B([A-Z])", r" \1", match_id.split("_")[-1])

    return home_team, away_team

In [7]:
def calculate_match_projections(home_attack_rating, home_defence_rating, away_attack_rating, away_defence_rating):
    
    home_projected_score = (home_attack_rating + away_defence_rating)*0.5
    away_projected_score = (home_defence_rating + away_attack_rating)*0.5
    projected_margin = home_projected_score - away_projected_score
    
    return home_projected_score, away_projected_score, projected_margin

In [8]:
def get_match_ratings(team_summary, home_team, away_team):
    
    match_ratings_dict = {'home_attack_rating':team_summary.loc[home_team]['exp_vaep_value_pergame_F'],
                          'home_defence_rating':team_summary.loc[home_team]['exp_vaep_value_pergame_A'],
                          'away_attack_rating':team_summary.loc[away_team]['exp_vaep_value_pergame_F'],
                          'away_defence_rating':team_summary.loc[away_team]['exp_vaep_value_pergame_A']}
    
    return match_ratings_dict

In [9]:
def calculate_projected_probabilities(home_projected_score, away_projected_score, score_std=25):
    
    projected_margin = home_projected_score - away_projected_score
    projected_margin_distribution = stats.norm(loc = projected_margin, scale = score_std)
    
    away_team_win_probability = projected_margin_distribution.cdf(0)
    draw_probability = projected_margin_distribution.pdf(0)
    home_team_win_probability = 1 - away_team_win_probability - draw_probability
    
    return home_team_win_probability, draw_probability, away_team_win_probability
    

In [10]:
def get_match_prediction_metrics(projected_margin, margin):
    
    mae = abs(projected_margin - margin)
    mse = ((projected_margin)**2 + (margin)**2)
    rmse = ((projected_margin)**2 + (margin)**2)**0.5
    
    return mae, mse, rmse

In [11]:
def get_match_result(data, match_id):
    
    home_team, away_team = get_teams(match_id)
    match_player_stats = data[data['Match_ID'] == match_id]
    match_score = match_player_stats.groupby('Team').sum()[['Score', 'xScore']]
    home_score, home_xscore = match_score.loc[home_team]['Score'], match_score.loc[home_team]['xScore']
    away_score, away_xscore = match_score.loc[away_team]['Score'], match_score.loc[away_team]['xScore']
    margin, xmargin = home_score - away_score, home_xscore - away_xscore
    
    return home_score, away_score, margin

In [12]:
def calculate_match_projection_error(player_stats, match_id, match_ratings_dict, score_std = 25):
    
    home_projected_score, away_projected_score, projected_margin = calculate_match_projections(match_ratings_dict['home_attack_rating'], 
                                                                             match_ratings_dict['home_defence_rating'], 
                                                                             match_ratings_dict['away_attack_rating'], 
                                                                             match_ratings_dict['away_defence_rating'])
    
    home_team_win_probability, draw_probability, away_team_win_probability = calculate_projected_probabilities(home_projected_score, away_projected_score, score_std)
    home_score, away_score, margin = get_match_result(player_stats, match_id)
    
    mae, mse, rmse = get_match_prediction_metrics(projected_margin, margin)
    
    return mae, mse, rmse

In [13]:
def get_latest_rounds(round_id, player_stats, rounds):
    
    all_rounds = sorted(list(set(player_stats['Round_ID'])))
    rating_rounds = all_rounds[all_rounds.index(round_id)-rounds:all_rounds.index(round_id)]
    player_stats_latest = player_stats[player_stats['Round_ID'].isin(rating_rounds)]
    
    return player_stats_latest

In [14]:
def update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae):
    
    previous_round_id = get_previous_round_id(round_id)
    player_stats_latest = get_latest_rounds(round_id, player_stats, rounds)
    team_summary = create_team_summary(player_stats_latest, rounds = rounds)
    team_summary = team_summary[['exp_vaep_value_pergame_F', 'exp_vaep_value_pergame_A', 'exp_vaep_value_pergame_diff']]

    round_projection_dict[round_id] = create_round_match_projections(player_stats, team_summary, round_id)
    round_mae[round_id] = np.mean(np.array(list({v['MAE'] for (k, v) in round_projection_dict[round_id].items()})))
    print("Round MAE: {}".format(round_mae[round_id]))
    print("Overall MAE: {}".format(np.mean(np.array(list(round_mae.values())))))
    return team_summary.sort_values('exp_vaep_value_pergame_diff', ascending = False)

In [15]:
def create_round_match_projections(player_stats, team_summary, round_id):
    
    player_stats_round = player_stats[player_stats['Round_ID'] == round_id]
    match_id_list = list(player_stats_round['Match_ID'].unique())

    match_projection_dict = {}
    for match_id in match_id_list:
        home_team, away_team = get_teams(match_id)
        match_ratings_dict = get_match_ratings(team_summary, home_team, away_team)
        home_projected_score, away_projected_score, projected_margin = calculate_match_projections(match_ratings_dict['home_attack_rating'], 
                                                                                                    match_ratings_dict['home_defence_rating'], 
                                                                                                    match_ratings_dict['away_attack_rating'], 
                                                                                                    match_ratings_dict['away_defence_rating'])
        home_team_win_probability, draw_probability, away_team_win_probability = calculate_projected_probabilities(home_projected_score, away_projected_score, score_std)
        home_score, away_score, margin = get_match_result(player_stats_round, match_id)
        mae, mse, rmse = calculate_match_projection_error(player_stats_round, match_id, match_ratings_dict, score_std = 25)

        match_projection_dict[match_id] = {}
        if home_projected_score > away_projected_score:
            match_projection_dict[match_id]['tip'] = home_team
            match_projection_dict[match_id]['projected_prob'] = home_team_win_probability
        else:
            match_projection_dict[match_id]['tip'] = away_team
            match_projection_dict[match_id]['projected_prob'] = away_team_win_probability
        match_projection_dict[match_id]['projected_margin'] = abs(projected_margin)
        if home_score > away_score:
            match_projection_dict[match_id]['result'] = home_team
        else:
            match_projection_dict[match_id]['result'] = away_team
        match_projection_dict[match_id]['actual_margin'] = abs(margin)   
        match_projection_dict[match_id]['MAE'] = mae
    
    return match_projection_dict
    

In [16]:
score_std = get_score_standard_deviation(player_stats)

In [17]:
round_projection_dict = {}
round_mae = {}
rounds = 10

202301

In [19]:
round_id = '202301'
player_stats_latest = player_stats[player_stats['Season'] == 2022]
team_summary = create_team_summary(player_stats_latest, rounds = rounds)
team_summary = team_summary[['exp_vaep_value_pergame_F', 'exp_vaep_value_pergame_A', 'exp_vaep_value_pergame_diff']]

round_projection_dict[round_id] = create_round_match_projections(player_stats, team_summary, round_id)
round_mae[round_id] = np.mean(np.array(list({v['MAE'] for (k, v) in round_projection_dict[round_id].items()})))
print("Round MAE: {}".format(round_mae[round_id]))
print("Overall MAE: {}".format(np.mean(np.array(list(round_mae.values())))))
team_summary.sort_values('exp_vaep_value_pergame_diff', ascending = False)

Round MAE: 30.9059044426566
Overall MAE: 30.9059044426566


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Geelong,121.9777,82.8621,39.1156
Richmond,131.8787,112.2374,19.6413
Sydney,123.2003,104.7422,18.4581
Port Adelaide,115.9103,102.4553,13.4549
Melbourne,119.3971,109.4337,9.9634
Western Bulldogs,109.3955,101.5918,7.8037
Brisbane Lions,116.5698,114.6918,1.8780
Fremantle,107.4655,109.4248,-1.9593
Carlton,105.3546,109.0419,-3.6873
St Kilda,93.6377,98.6196,-4.9819


202302

In [20]:
round_id = '202302'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 35.74120399780804
Overall MAE: 33.323554220232324


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Geelong,124.7766,91.7470,33.0296
Port Adelaide,128.9105,101.3077,27.6029
Sydney,125.8993,104.4167,21.4827
Richmond,132.2227,113.7513,18.4714
Melbourne,122.7672,107.7734,14.9937
Western Bulldogs,110.4462,107.2734,3.1727
Fremantle,110.0481,110.2557,-0.2076
Carlton,114.5739,115.4642,-0.8903
St Kilda,99.2764,104.6213,-5.3449
Collingwood,112.3491,118.6902,-6.3412


202303

In [21]:
round_id = '202303'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 27.335637380163387
Overall MAE: 31.327581940209345


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Geelong,123.1681,92.0849,31.0832
Sydney,123.5245,101.4756,22.0489
Port Adelaide,124.1248,103.6571,20.4677
Richmond,136.4901,118.5832,17.9069
Melbourne,120.3032,107.1862,13.1170
Fremantle,110.5797,109.1249,1.4548
Collingwood,115.9370,116.4544,-0.5174
Western Bulldogs,107.3290,109.0252,-1.6962
St Kilda,104.4450,107.8744,-3.4294
Carlton,112.0409,115.5783,-3.5374


202304

In [22]:
round_id = '202304'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 31.594983320067975
Overall MAE: 31.394432285174005


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Geelong,121.1497,93.7811,27.3686
Port Adelaide,126.7299,108.9881,17.7418
Richmond,131.9445,116.5060,15.4385
Sydney,120.1725,107.5867,12.5859
Melbourne,119.5246,108.7604,10.7642
Fremantle,116.7095,109.3920,7.3175
Carlton,120.4504,116.6358,3.8146
Western Bulldogs,110.0005,108.0723,1.9282
Collingwood,115.8127,114.4177,1.3951
Greater Western Sydney,117.4618,122.1467,-4.6849


202305

In [23]:
round_id = '202305'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 24.5362252491962
Overall MAE: 30.022790877978444


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Geelong,123.6590,94.2752,29.3839
Port Adelaide,130.1948,107.6302,22.5646
Melbourne,117.7806,104.1680,13.6126
Sydney,119.6853,109.3068,10.3785
Richmond,131.2757,123.1035,8.1722
Carlton,120.5223,112.3713,8.1510
St Kilda,115.7853,108.6041,7.1812
Fremantle,115.7424,110.1355,5.6069
Western Bulldogs,113.9880,111.3622,2.6258
Collingwood,114.9040,113.3895,1.5145


202306

In [24]:
round_id = '202306'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 30.189168428607577
Overall MAE: 30.05052046974997


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Geelong,122.8004,95.6983,27.1020
Port Adelaide,128.3674,110.2629,18.1045
St Kilda,121.4818,109.5306,11.9512
Essendon,134.2646,124.3983,9.8663
Melbourne,119.3882,109.9163,9.4718
Sydney,118.0520,110.2773,7.7747
Collingwood,116.5742,110.9692,5.6050
Adelaide,124.6915,120.9890,3.7024
Richmond,132.1659,128.8543,3.3115
Fremantle,116.3871,114.3377,2.0495


202307

In [25]:
round_id = '202307'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 34.3972033669078
Overall MAE: 30.671475169343946


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Geelong,121.4133,97.3225,24.0908
St Kilda,124.2300,107.4473,16.7827
Essendon,132.8689,116.7810,16.0879
Port Adelaide,131.8972,118.5110,13.3861
Adelaide,133.0057,119.6507,13.3550
Collingwood,118.3835,108.9879,9.3956
Melbourne,117.4469,111.1075,6.3394
Western Bulldogs,117.0308,119.4972,-2.4664
Sydney,113.3896,116.0075,-2.6179
Carlton,117.6251,121.2578,-3.6327


202308

In [26]:
round_id = '202308'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 21.262294432725135
Overall MAE: 29.495327577266593


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Geelong,123.7663,97.8339,25.9325
Adelaide,129.0566,115.0717,13.9849
St Kilda,121.0471,107.2730,13.7740
Melbourne,120.4225,106.8637,13.5587
Port Adelaide,128.2550,116.5998,11.6552
Carlton,121.6792,111.7814,9.8978
Essendon,129.7134,119.9110,9.8024
Collingwood,116.0610,107.4689,8.5920
Western Bulldogs,114.5499,114.2141,0.3357
Sydney,114.1953,115.7373,-1.5420


202309

In [27]:
round_id = '202309'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 40.48643312949962
Overall MAE: 30.71656152751471


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Geelong,123.8848,98.2784,25.6064
Port Adelaide,129.1280,113.3213,15.8067
St Kilda,119.0907,105.3687,13.7220
Melbourne,122.1377,110.9052,11.2325
Adelaide,125.7094,116.3067,9.4028
Collingwood,114.5663,107.0672,7.4990
Carlton,120.3660,114.0055,6.3605
Essendon,124.7956,121.8269,2.9687
Western Bulldogs,112.8246,112.5315,0.2931
Richmond,122.3875,122.3996,-0.0121


202310

In [28]:
round_id = '202310'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 31.402608658836634
Overall MAE: 30.7851662406469


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Geelong,121.6921,99.0824,22.6098
Port Adelaide,130.8866,110.6298,20.2568
Melbourne,121.5747,107.1785,14.3963
Collingwood,115.0377,102.8739,12.1638
Adelaide,124.6441,112.5121,12.1319
Brisbane Lions,118.2840,109.1409,9.1432
St Kilda,114.4738,106.7127,7.7611
Gold Coast,117.5068,111.6530,5.8539
Carlton,118.7500,113.5837,5.1664
Western Bulldogs,112.5340,111.7860,0.7479


202311

In [29]:
round_id = '202311'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 19.565182170566604
Overall MAE: 29.76516768882142


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Port Adelaide,130.4375,109.8517,20.5858
Geelong,119.2973,104.2894,15.0079
Collingwood,115.4480,102.2730,13.1750
Brisbane Lions,119.2626,106.5596,12.7030
Melbourne,119.7021,109.1001,10.6020
St Kilda,117.2679,106.6995,10.5684
Adelaide,120.8492,114.7561,6.0931
Western Bulldogs,114.7758,109.2770,5.4988
Carlton,116.6217,114.0984,2.5233
Sydney,113.6561,112.3978,1.2583


202312

In [30]:
round_id = '202312'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 20.432043958353827
Overall MAE: 28.987407377949122


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Brisbane Lions,123.0767,100.0767,23.0000
Geelong,117.4043,101.7386,15.6657
Port Adelaide,124.6084,110.0928,14.5156
Collingwood,111.8774,99.6859,12.1915
Western Bulldogs,118.2020,108.1050,10.0970
Melbourne,117.9449,109.3160,8.6288
St Kilda,115.7592,107.6798,8.0794
Adelaide,120.7125,114.4903,6.2222
Gold Coast,116.4186,111.7024,4.7162
Carlton,114.8925,110.7952,4.0972


202313

In [31]:
round_id = '202313'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 28.89171597812939
Overall MAE: 28.98004650103991


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Brisbane Lions,123.6476,100.0885,23.5591
Port Adelaide,133.1460,111.5724,21.5736
Geelong,118.7383,101.8387,16.8996
Melbourne,119.3497,104.9926,14.3572
Collingwood,114.6281,100.6005,14.0277
Western Bulldogs,119.6173,105.6205,13.9969
Gold Coast,117.8507,110.6334,7.2173
St Kilda,113.7345,108.4663,5.2682
Adelaide,120.1378,115.2398,4.8980
Carlton,110.9660,112.8212,-1.8552


202314

In [32]:
round_id = '202314'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 32.585329787293006
Overall MAE: 29.237566735772276


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Port Adelaide,133.5001,107.4913,26.0088
Brisbane Lions,126.2874,103.5165,22.7709
Geelong,122.7251,102.6001,20.1249
Melbourne,120.4355,104.7881,15.6474
Adelaide,124.6274,112.5568,12.0706
Western Bulldogs,120.2427,109.1315,11.1112
Collingwood,113.6245,104.9179,8.7066
Gold Coast,120.3913,113.7197,6.6716
St Kilda,110.2200,103.7609,6.4590
Essendon,116.4106,117.9815,-1.5710


202315

In [33]:
round_id = '202315'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 45.71939576982021
Overall MAE: 30.33635533804214


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Port Adelaide,135.9979,105.2635,30.7344
Brisbane Lions,126.8367,101.1651,25.6716
Melbourne,120.1716,107.2517,12.9199
Western Bulldogs,118.4168,106.0521,12.3647
Collingwood,113.9306,102.0940,11.8366
Adelaide,125.1005,114.2047,10.8958
Geelong,118.3790,109.7599,8.6191
Gold Coast,120.5471,116.0282,4.5189
Carlton,112.4929,110.9129,1.5801
Richmond,112.2276,111.3609,0.8667


202316

In [34]:
round_id = '202316'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 37.538164294727444
Overall MAE: 30.78646839783497


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Port Adelaide,136.0845,104.7420,31.3425
Brisbane Lions,124.3258,99.9159,24.4099
Western Bulldogs,119.3568,102.8114,16.5454
Melbourne,120.0662,104.4232,15.6430
Gold Coast,124.4060,111.6006,12.8054
Collingwood,113.3914,102.5108,10.8806
Carlton,113.0077,105.7144,7.2933
Geelong,117.9132,112.6081,5.3051
Adelaide,119.3389,114.3400,4.9989
Richmond,112.2276,111.3609,0.8667


202317

In [35]:
round_id = '202317'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 23.9600398629836
Overall MAE: 30.38491377813783


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Brisbane Lions,126.3141,97.3366,28.9775
Port Adelaide,131.4907,105.6396,25.8511
Melbourne,119.9198,99.9427,19.9771
Collingwood,117.4714,100.5343,16.9371
Carlton,115.3516,102.5792,12.7724
Adelaide,122.2731,109.8137,12.4595
Sydney,115.5670,104.6150,10.9520
Western Bulldogs,115.0890,104.3129,10.7761
St Kilda,106.8225,102.7558,4.0666
Gold Coast,119.8475,118.9862,0.8613


202318

In [36]:
round_id = '202318'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 29.732271069086885
Overall MAE: 30.348655849857224


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Brisbane Lions,128.7390,95.4175,33.3215
Port Adelaide,135.3091,105.6904,29.6187
Collingwood,118.9926,100.0620,18.9306
Melbourne,114.5434,101.6250,12.9183
Western Bulldogs,115.0842,106.7942,8.2900
Adelaide,123.6173,116.2820,7.3353
Carlton,113.3527,106.8545,6.4981
St Kilda,107.0202,101.3697,5.6505
Sydney,110.5405,105.7925,4.7480
Greater Western Sydney,110.3546,108.6876,1.6671


202319

In [37]:
round_id = '202319'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 16.56252368106699
Overall MAE: 29.623069946236683


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Brisbane Lions,126.1553,93.7608,32.3946
Port Adelaide,133.9568,112.2668,21.6899
Collingwood,121.0771,101.5233,19.5538
Melbourne,110.1686,98.3108,11.8577
Carlton,117.6177,106.1316,11.4861
Adelaide,124.2088,115.7330,8.4759
Western Bulldogs,117.3660,109.7343,7.6317
Sydney,114.7477,108.0010,6.7467
Geelong,113.0863,108.3571,4.7291
Greater Western Sydney,112.4943,109.4493,3.0450


202320

In [38]:
round_id = '202320'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 25.083430264370126
Overall MAE: 29.396087962143355


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Brisbane Lions,120.4862,92.2657,28.2205
Carlton,121.9958,102.4502,19.5456
Port Adelaide,131.9383,113.1795,18.7588
Collingwood,118.3264,105.1477,13.1787
Greater Western Sydney,118.8288,106.6133,12.2155
Sydney,116.7740,105.5703,11.2037
Western Bulldogs,119.3843,109.4093,9.9750
Melbourne,110.5383,102.6931,7.8452
St Kilda,108.3735,101.2044,7.1690
Adelaide,124.2849,119.9822,4.3027


202321

In [39]:
round_id = '202321'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 22.48930304515638
Overall MAE: 29.067193442286833


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Carlton,123.2327,102.3278,20.9049
Brisbane Lions,118.8919,97.9885,20.9034
Greater Western Sydney,119.3010,103.2220,16.0790
Melbourne,116.2886,101.6900,14.5985
Adelaide,132.8436,119.1068,13.7369
Port Adelaide,132.0137,119.9435,12.0703
Collingwood,118.2040,106.3846,11.8195
Sydney,121.5443,110.5486,10.9957
St Kilda,108.2648,101.6961,6.5687
Western Bulldogs,116.5117,112.0924,4.4193


202322

In [40]:
round_id = '202322'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 24.02007047096134
Overall MAE: 28.837778761772032


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Carlton,122.8064,99.9572,22.8491
Brisbane Lions,116.7188,95.4721,21.2467
Adelaide,134.9781,115.5315,19.4466
Greater Western Sydney,118.3891,103.1571,15.2319
Melbourne,116.9582,102.1041,14.8541
Sydney,121.5636,111.4625,10.1011
Western Bulldogs,121.4984,111.9916,9.5069
Port Adelaide,132.9116,124.2638,8.6478
Collingwood,116.9555,110.0456,6.9100
St Kilda,105.6591,98.7712,6.8880


202323

In [41]:
round_id = '202323'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 32.70128954074598
Overall MAE: 29.005757491292638


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Carlton,126.1233,99.6131,26.5102
Adelaide,135.6372,112.5185,23.1187
Brisbane Lions,116.6578,99.4613,17.1966
Greater Western Sydney,117.4966,103.7822,13.7144
Melbourne,116.6140,105.4211,11.1930
Sydney,122.6422,111.8105,10.8317
Port Adelaide,129.7329,121.3605,8.3724
St Kilda,105.9643,98.3680,7.5963
Western Bulldogs,121.2826,113.8828,7.3998
Geelong,111.4686,106.1801,5.2884


202324